In [61]:
# Import libraries
using Turing
using LinearAlgebra
using Distributions
using MultivariateStats
import MultivariateStats: reconstruct
using GaussianProcesses
using StatsBase
using Statistics
using Suppressor
using JLD2
using CSV
using DataFrames, DataFramesMeta
using SplitApplyCombine
using KernelFunctions
using MCMCChains
using PyCall
using PyPlot
using Printf
import PyCall.pyfunction


# Access the matplotlib module
matplotlib = pyimport("matplotlib")
seaborn = pyimport("seaborn")
os = pyimport("os")
pyimport("scienceplots")
np = pyimport("numpy")

plt.style.use(["default","science","no-latex"])
using StatsPlots

seaborn.color_palette("colorblind")

PyCall.pygui(:tk)

include("../Utils/scale_utils.jl")
using .ScaleUtils

include("../Utils/gp_utils.jl")
using .GPUtils

# Set a seed for reproducibility
using Random
Random.seed!(11);

FONTSIZE=20.5;

In [62]:
matplotlib_backends_pdf = pyimport("matplotlib.backends.backend_pdf")
PdfPages = matplotlib_backends_pdf.PdfPages

PyObject <class 'matplotlib.backends.backend_pdf.PdfPages'>

In [63]:
# #Loading all calibrated posteriors from a given realization of future observations, storing them into a dictionary

# #Note that this function assumes a specific file structure of the HPC_Output_Folder:
#     # Example:   Root/output_R1_job808_task0/chains_dir/2030_chain.jld2

# sample_size = 5000

# function load_data(realization, HPC_Results_Folder, saveout_directory)
#     post_data = Dict{Int64, Matrix{Float64}}()

#     for dir in readdir(HPC_Results_Folder)
#         if occursin("_R$(realization)_", dir)
#             chain_path = joinpath(HPC_Results_Folder, dir, "chains_dir" )
#             #Read only the chains, not the generated quantities, for now
#             chain_string = only(readdir(chain_path))
#             #Get the current year
#             year = parse(Int, SubString(chain_string, 1:4))
#             #Load the chain from disc
#             current_chain = JLD2.load("$(chain_path)/$(chain_string)", "chain" )
#             #Convert the chain into a matrix
#             chain_matrix = get_params(current_chain)
#             θ_post = [chain_matrix.vmThresh;; chain_matrix.fricExp;; chain_matrix.mu_scale;; 
#                          chain_matrix.stiff_scale;; chain_matrix.gamma0;; chain_matrix.melt_flux]
#             #Add this matrix to the dicrtionary according to its last year of constraining observation

#             post_data[ year ] = θ_post
    
#         end
#     end
#     #Save this dictionary to disk
#     @save "$(saveout_directory)/R_$(realization)_Posterior_Dict.jld2" post_data
    
#     return nothing
# end

In [64]:
# #Define the path to where your HPC results are
# hpc_data_directory = "./MCMC_Outputs_Sanket_ReRuns_New_All" # "../../BrookhavenCode/ProjectNotebooks/MCMC_Outputs_Sanket"
# saveout_location = "./Data/Posterior_Data_New_All"

# #Realization numbers
# Realizations = [i for i in 1:100];
# # Realizations = [4, 15, 20, 25, 44, 47, 67, 81, 82, 89, 100]

# for r in Realizations
#     load_data(r, hpc_data_directory, saveout_location)
# end

In [65]:
# using Measures
# base = "/home/sjantre/Projects/Future_Learning/BrookhavenCode/MCMC_Outputs_Sanket_Failed_ReRuns_New_2"
# real = 67
# years = [2060, 2075, 2090, 2105]                       # <-- change freely
# want   = [:vmThresh, :fricExp, :mu_scale, :stiff_scale, :gamma0, :melt_flux]
# max_iter = 50_00                            # target draws per chain for plotting
# trace_outdir  = "./Plots/Trace_Plots"
# corner_outdir = "./Plots/Pairwise"

# # --- helpers ---
# function find_chain_path(base::AbstractString, realization::Int, year::Int)
#     dirs = filter(d -> occursin("_R$(realization)_", d), readdir(base))
#     for d in dirs
#         chain_dir = joinpath(base, d, "chains_dir")
#         isdir(chain_dir) || continue
#         files = readdir(chain_dir)
#         isempty(files) && continue
#         f = only(files)                           # one file per chains_dir
#         y = parse(Int, first(f, 4))               # first 4 chars are the year
#         y == year && return joinpath(chain_dir, f)
#     end
#     error("No chain file found for realization $realization, year $year in $base")
# end

# mkpath(trace_outdir); mkpath(corner_outdir)

# # load chains for requested years
# paths  = Dict(y => find_chain_path(base, real, y) for y in years)
# chains = Dict(y => JLD2.load(paths[y], "chain") for y in years)  # Dict{Int,Chains}

# chain_matrix(ch::Chains, vars) = begin
#     A = Array(ch[:, vars, :])                     # (iter, param, chain)
#     reshape(A, size(A,1) * size(A,3), size(A,2))  # -> (draws, params)
# end

# # # winsorize columns to robust limits
# # function winsorize_cols!(X; lo=0.005, hi=0.995)
# #     @assert 0.0 ≤ lo < hi ≤ 1.0
# #     for j in axes(X, 2)
# #         v = view(X, :, j)                      # SubArray view of column j
# #         qlo, qhi = quantile(v, (lo, hi))
# #         @. v = clamp(v, qlo, qhi)
# #     end
# #     return X
# # end


# for y in years
#     ch = chains[y]
#     pars = isempty(want) ? names(ch, :parameters) : intersect(want, names(ch, :parameters))

#     # thin to ~max_iter iterations
#     step    = max(1, cld(size(ch, 1), max_iter))
#     ch_thin = ch[1:step:end, pars, :]  # (iter, parameters, chain)

#     # — trace plot (renumber iterations for a clean x-axis) —
#     ch_plot = Chains(Array(ch_thin), collect(pars); start=1, thin=1)
#     p_trace = StatsPlots.plot(ch_plot; seriestype=:traceplot, legend=false,
#         title="Realization $real — Year $y (thinned ×$step)",
#         layout=(length(pars), 1), size=(1100, max(220*length(pars), 300)))
#     StatsPlots.savefig(p_trace, joinpath(trace_outdir, "R$(real)_$(y)_trace_thin_New_2.pdf"))

#     gr(size=(400,300))
#     # --- corner (pairwise) plot ---
#     if length(pars) ≥ 2
#         X = chain_matrix(ch_thin, pars)  # draws × params
#         cell = 380
#         n    = size(X, 2)

#         p_corr = corrplot(
#             X ,compact=true;
#             label = permutedims(String.(pars)),  # labels per column
#             grid  = true,
#             size  = (cell*n, cell*n)
#         )
#         StatsPlots.savefig(p_corr, joinpath(corner_outdir, "R$(real)_$(y)_corner_New_2.pdf"))
#     else
#         @info "Skipping corner plot for year $y (need ≥2 parameters)", pars
#     end
# end

In [66]:
################################################################################
################################################################################
################################################################################

# KDE PLOTS

################################################################################
################################################################################
################################################################################

In [67]:
# # PRIOR DISTRIBUTIONS

# sample_size = 5000

# vmThresh_vals = collect(np.linspace(80000+0.00001, 180000-0.00001, sample_size))
# fricExp_vals = collect(np.linspace(0.1+0.00001, 0.333-0.00001, sample_size))
# mu_scale_vals = collect(np.linspace(0.8+0.00001, 1.2-0.00001, sample_size))
# stiff_scale_vals = collect(np.linspace(0.8+0.00001, 1.2-0.00001, sample_size))
# gamma0_vals = collect(np.linspace(9620+0.0001, 471000-0.0001, sample_size))
# melt_flux_vals = collect(np.linspace(12+0.0001, 58-0.0001, sample_size))


# vmThresh_prior = collect(scipy.stats.truncnorm.pdf(vmThresh_vals, a=-3, b=3, loc=130000, scale=50000/3))
# fricExp_prior = collect(scipy.stats.trapezoid.pdf(fricExp_vals, c=0.05/0.233, d=0.18/0.233, loc=0.1, scale=0.233))
# mu_scale_prior = collect(scipy.stats.truncnorm.pdf(mu_scale_vals, a=-2, b=2, loc=1.0, scale=0.1))
# stiff_scale_prior = collect(scipy.stats.truncnorm.pdf(stiff_scale_vals, a=-2, b=2, loc=1.0, scale=0.1))
# trunc_norm_gamma0 = scipy.stats.truncnorm(a=np.log(9620)-10, b=np.log(471000)-10, loc=10, scale=1)
# gamma0_prior = collect(trunc_norm_gamma0.pdf(np.log(gamma0_vals)) ./ gamma0_vals)
# melt_flux_prior = collect(scipy.stats.truncnorm.pdf(melt_flux_vals, a=-2, b=2, loc=35, scale=11.5))

# θ_prior = [vmThresh_prior;; fricExp_prior;; mu_scale_prior;; 
#      stiff_scale_prior;; gamma0_prior;; melt_flux_prior];

#Loading original parameter data for scaling purposes
X_raw = CSV.read("../Data/Training_Data/Amery_Input_Parameters_Filtered.csv", DataFrame);
# 1) Grab all column‐names as Symbols
cols = Symbol.(names(X_raw))
# 2) Remove the index‐column symbol
cols = filter(c -> c != :Column1, cols)
# 3) Now call get_scaled_matrix on the remaining columns
X_scaled_t, X_scalers, X_mins, X_maxs = ScaleUtils.get_scaled_matrix(X_raw, cols);

In [68]:
function unscale_params(params)
    
    unscaled_θ_1 = ( params[1] * (X_maxs[:vmThresh] - X_mins[:vmThresh]) ) + X_mins[:vmThresh]
    unscaled_θ_2 = ( params[2] * (X_maxs[:fricExp] - X_mins[:fricExp]) ) + X_mins[:fricExp]
    unscaled_θ_3 = ( params[3] * (X_maxs[:mu_scale] - X_mins[:mu_scale]) ) + X_mins[:mu_scale]
    unscaled_θ_4 = ( params[4] * (X_maxs[:stiff_scale] - X_mins[:stiff_scale]) ) + X_mins[:stiff_scale]
    unscaled_θ_5 = ( params[5] * (X_maxs[:gamma0] - X_mins[:gamma0]) ) + X_mins[:gamma0]
    unscaled_θ_6 = ( params[6] * (X_maxs[:melt_flux] - X_mins[:melt_flux]) ) + X_mins[:melt_flux];
    unscaled_thetas = [unscaled_θ_1
                        unscaled_θ_2
                        unscaled_θ_3
                        unscaled_θ_4
                        unscaled_θ_5
                        unscaled_θ_6];
    return unscaled_thetas
    
end

unscale_params (generic function with 1 method)

In [69]:
# using KernelDensity

# @inline function _trapz(x::AbstractVector{<:Real}, y::AbstractVector{<:Real})
#     @assert length(x) == length(y) && length(x) ≥ 2
#     s = zero(eltype(x))
#     @inbounds for i in 1:length(x)-1
#         s += (x[i+1] - x[i]) * (y[i+1] + y[i])
#     end
#     return s/2
# end

# function rescale_kde(data, a, b; npoints=1024)
#     # # Fit a KDE to the data
#     # x = sort(float.(collect(skipmissing(data))))
#     # lo, hi = minmax(float(a), float(b))
#     # if lo == hi
#     #     δ = max(eps(lo)*100, 1e-9)
#     #     lo -= δ; hi += δ
#     # end
#     # kd = kde(x)
#     # xs = range(lo, hi; length=npoints)
#     # ys = pdf(kd, xs)
#     # # normalize on [lo,hi]
#     # mass = _trapz(xs, ys)
#     # ys = mass > 0 ? ys ./ mass : zeros(length(xs))
#     # return collect(xs), ys

#     # Fit a KDE to the data
#     data_sorted = sort(data)
#     kde = KernelDensity.kde(data_sorted)
#     # Evaluate the KDE at the grid points
#     kde_values = pdf(kde, data_sorted)
#     # Compute the CDF
#     cdf = cumsum(kde_values)
#     # Normalize the CDF
#     cdf ./= cdf[end]
#     # Calculate the probability that the variable is between a and b
#     last = searchsortedfirst(data_sorted, b)
#     if  last > size(data_sorted)[1]
#         last = last - 1
#     end
#     prob_a_to_b = cdf[last] - cdf[searchsortedfirst(data_sorted, a)]
#     # Filter all the data to be less than b or greater than a
#     mask = a .<= data_sorted .<= b
#     filtered_data = data_sorted[mask]

#     # Rescale the KDE values within [a, b] and zero out values outside
#     #Since we're looking at values between A and B and we want to get a PDF for those values,
#     # the curve must integrate to 1 to be valid. Rescaling below allows this to happen.
#     kde_values_rescaled = pdf(kde, filtered_data) / prob_a_to_b

# #     return data_sorted, kde_values_rescaled
#     return filtered_data, kde_values_rescaled
# end

In [109]:
#Previous paper's Calibrated Posteriors
θ_posterior_2015 = np.load("../Data/Training_Data/posterior_samples_All_Combined.npy");

In [110]:
# #Just a helper function that maps a tuple to a single index for a 2x3 grid of plots
function tuple_to_idx(x,y)
    return ( (3*x - 3) + y )
end

# #Function that plots a specific parameter's distribution from the posteriors dervied in Sanket's Paper
# function plot_previous(previous_post, ax, var_idx)

#         uppers_and_lowers = Dict([  1 => (80000,180000), 2 => (0.1,0.333),
#                             3 => (0.8,1.2), 4 => (0.8,1.2),
#                             5 => (np.log(9620),np.log(471000)), 6 => (12,58)  ])
    
#         current_params = previous_post[:, var_idx]
#         param_mean = mean(current_params)
#         #Gamma_0 requires a log scale
#         if var_idx == 5
#             x, y = rescale_kde(np.log(current_params), uppers_and_lowers[var_idx][1], uppers_and_lowers[var_idx][2] )
#             ax.plot(x, y,color="red", label = 2015, lw=1.75)
#             #ax.axvline(x=param_mean) #ymin=0, ymax=ax.get_ylim()[2], color = "red", linestyle="dashed"


#         else
#             x, y = rescale_kde(current_params, uppers_and_lowers[var_idx][1], uppers_and_lowers[var_idx][2] )
#             ax.plot(x, y, color="red", label = "Present-day (2015) posterior", lw=1.75)
#             #ax.axvline(x=param_mean) #ymin=0, ymax=ax.get_ylim()[2], color = "red", linestyle="dashed"
                
#         end
# end
    
    
# #Function to plot all years worth of a specific paramerters distributions
# function plot_one_var_posterior(post_dict, years, ax, var_idx, cmaplist)

#     uppers_and_lowers = Dict([  1 => (80000,180000), 2 => (0.1,0.333),
#                                 3 => (0.8,1.2), 4 => (0.8,1.2),
#                                 5 => (np.log(9620),np.log(471000)), 6 => (12,58)  ])
    
#     for i in 1:length(years)
#         #Get all n_obs posteriors for one variable
#         #Below is line 41
#         current_params = post_dict[years[i]][:,var_idx]

#         x = filter(isfinite, float.(current_params))
#         a, b = uppers_and_lowers[var_idx][1], uppers_and_lowers[var_idx][2]
#         # println("a=$(a) b=$(b)  extrema(x)=$(extrema(x))  var≈", var(x))
        
#         #Gamma_0 requires a log scale
#         if var_idx == 5
#                 x, y = rescale_kde(np.log(current_params), uppers_and_lowers[var_idx][1], uppers_and_lowers[var_idx][2] )
#                 ax.plot(x, y, lw=1.8, color= cmaplist(norm(years[i])))
#         else
#                 x, y = rescale_kde(current_params, uppers_and_lowers[var_idx][1], uppers_and_lowers[var_idx][2] )
#                 ax.plot(x, y, lw=1.8, color= cmaplist(norm(years[i])) )
#         end
#     end       
# end

tuple_to_idx (generic function with 1 method)

In [111]:
# #Function that plots all posterior distribtuons of all parameters

# function plot_all_Rs_KDE_pdf(real, post_dict, years, cmaplist, prev, pdf, theta)
   
#             fig, ax = PyPlot.subplots(nrows=2, ncols=3, figsize=(18, 11), dpi=300,
#                              gridspec_kw=Dict("height_ratios"=> [1, 1], "width_ratios" => [1, 1, 1],
#                                               "wspace"=> 0.175, "hspace"=> 0.25))

#             # fig.suptitle("Realization $real"; fontsize=30, fontweight="bold", y=0.96)
    
#             title_dict = Dict([
#                 1 => L"\sigma_{max}", 2 => L"q" , 3 => L"C_{\mu}" ,
#                 4 => L"C_{\phi}", 5 => L"\log(\gamma_0)", 6 => L"\overline{m}" ]) 
    
#             tick_dict = Dict([
#                     1 => [80000,100000,120000,140000,160000,180000], 2 => [0.1,0.15,0.215,0.28,0.333], 3 => [0.8,0.9,1,1.1,1.2],
#                     4 => [0.8,0.9,1,1.1,1.2], 5 => [9.17,10,11,12,13.06], 6 => [12,20,30,40,50,58] ]) 
            
#             lim_dict = Dict([
#                     1 => ([0.75e5,1.85e5], [0.01e-5,4.15e-5]), 2 => ([0.093,0.34], [0.01,8] ), 3 => ([0.79,1.21], [0.01,6.45] ),
#                     4 => ([0.79,1.21], [0.01,6.85] ), 5 => ([5,13.2], [0.001,1] ), 6 => ( [10.5,59.5], [0.0001,0.0465] )  ]) 
        
#             for i in 1:2
#                 for j in 1:3
#                     var_idx = tuple_to_idx(i,j)
#                     #Plot the posteriors
#                     plot_one_var_posterior(post_dict, years, ax[i,j], var_idx , cmaplist )
#                     #Plot the 2015 relx posteriors
#                     plot_previous(prev, ax[i,j], var_idx)
    
#                     #Plot formatting
#                     ax[i,j].set_title(title_dict[var_idx],fontsize=1.2*FONTSIZE, pad=10)    #("von Mises threshold (Pa)")        
#                     ax[i,j].ticklabel_format(style="sci", scilimits=(-2,2), useMathText=true)
#                     ax[i,j].tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
#                     ax[i,j].xaxis.offsetText.set_fontsize(FONTSIZE*0.8)
#                     ax[i,j].yaxis.offsetText.set_fontsize(FONTSIZE*0.8)
#                     ax[i,j].set_xticks(tick_dict[var_idx])
#                     ax[i,j].locator_params(tight=true, nbins=6)
#                     #ax[i,j].set_xlim(lim_dict[var_idx][1])
#                     #ax[i,j].set_ylim(lim_dict[var_idx][2])
#                     ax[i,j].grid(true, alpha=0.5, zorder=1)
                
#                 end
#             end
        
#         # --- Plotting Priors --- #
#             ax[1,1].plot(vmThresh_vals, θ_prior[:,1], lw=1.5, ls="dashed", label="Prior", color = "green")
#             ax[1,2].plot(fricExp_vals, θ_prior[:,2], lw=1.5, ls="dashed", color = "green")
#             ax[1,3].plot(mu_scale_vals, θ_prior[:,3], lw=1.5, ls="dashed", color = "green")
#             ax[2,1].plot(stiff_scale_vals, θ_prior[:,4], lw=1.5, ls="dashed", color = "green")
#             ax[2,2].plot(np.log(gamma0_vals), θ_prior[:,5].*gamma0_vals, lw=1.5, linestyle="dashed", color = "green")
#             ax[2,3].plot(melt_flux_vals, θ_prior[:,6], lw=1.5, ls="dashed", color = "green")
        

#             ax[1,1].axvline(x = theta[1], lw = 1.75, color = "black", ls = "dashdot", label = "Sample from present-day posterior")
#             ax[1,2].axvline(x = theta[2], lw = 1.75, color = "black", ls = "dashdot")
#             ax[1,3].axvline(x = theta[3], lw = 1.75, color = "black", ls = "dashdot")
#             ax[2,1].axvline(x = theta[4], lw = 1.75, color = "black", ls = "dashdot")
#             ax[2,2].axvline(x = np.log(theta[5]), lw = 1.75, color = "black", ls = "dashdot")
#             ax[2,3].axvline(x = theta[6], lw = 1.75, color = "black", ls = "dashdot")

        
    
#         # --- Colorbar Implementation --- #
#             scalar_mappable = PyPlot.matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)
#             scalar_mappable.set_array(max_years)
#             #plt.tight_layout()
#             cbar_ax = fig.add_axes([0.928, 0.25, 0.02, 0.5])  # Position: [left, bottom, width, height]
#             color_bar = plt.colorbar(scalar_mappable, cax=cbar_ax, pad = 0.1)
    
#             color_bar.set_ticks(max_years)
#             color_bar.ax.set_title("Analysis \nyear", fontsize=FONTSIZE*0.8, pad=20)
#             color_bar.set_ticklabels(["$(y)" for y in max_years])
#             color_bar.ax.tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
    
#             handles, labels = ax[1,1].get_legend_handles_labels()
#             desired_indices = [1,2,3]
#             sorted_handles = [handles[i] for i in desired_indices]
#             sorted_labels = [labels[i] for i in desired_indices]
#             fig.legend(sorted_handles, sorted_labels, loc="lower center", ncol=6, fontsize=FONTSIZE,
#                        bbox_to_anchor=(0.512, 0.0), bbox_transform=PyPlot.gcf().transFigure,
#                        frameon="True", framealpha=1)  
            
#             pdf.savefig(fig)   
#             PyPlot.close(fig)  
# end

In [112]:
# using MCMCDiagnosticTools
# #Define plot saveout location
# figure_save_path = "./Plots/KDE_Plots/KDE_plots_Realization_test.pdf"
# #Where your posteriors are located
# posteriors_directory = "./Data/Posterior_Data_New_All" 
# #Where your generating parameters sets are located
# future_obs_directory = "./IntermediateData/emulator_pred/0.8244943887658679-data"

# max_years = collect(range(2030, step=45, length=7))
# yr_min, yr_max = minimum(max_years), maximum(max_years)
# norm = PyPlot.matplotlib.colors.Normalize(vmin=yr_min, vmax=yr_max)
# cmap = PyPlot.cm.get_cmap("cool")


# pdf_pages = PdfPages(figure_save_path)

# #Realization numbers
# realizations = [58] #[i for i in 1:100];
# r_filtered = [string(i) for i in realizations];

# try
#     for realization in r_filtered
#         ### Enter correct posterior dict in line below
#         chosen_dict = JLD2.load("$(posteriors_directory)/R_$(realization)_Posterior_Dict.jld2", "post_data")
#         θ = JLD2.load("$(future_obs_directory)/$(realization)_emulator_data.jld2","θ");
#         θ_un = unscale_params(θ)
#         @show size(chosen_dict[2090])
#         A = chosen_dict[2090]  # size: iters × params  (check this!)
#         pnames = [:vmThresh, :fricExp, :mu_scale, :stiff_scale, :gamma0, :melt_flux]
        
#         A3 = reshape(A, size(A,1), size(A,2), 1)   # (iter, param, chain)
#         ch = Chains(A3, pnames; start=1, thin=1)   # <-- names are positional here
        
#         # @show MCMCDiagnosticTools.ess(ch)               # bulk ESS per param
        
#         plot_all_Rs_KDE_pdf(realization, chosen_dict, max_years, cmap, θ_posterior_2015, pdf_pages, θ_un)
#         println("Done with realization: $(realization) ")
#     end
        

# finally 
#     # --- Always close the PdfPages object to finalize the PDF file ---
#     # This ensures the PDF is properly written to disk and not corrupted.
#     pdf_pages.close()
# end

In [113]:
########################################
########################################
# ESS
########################################
########################################

In [114]:
# using JLD2, MCMCChains, DataFrames, Printf

# posteriors_directory = "./Data/Posterior_Data_New_All"

# years = collect(range(2030, step=15, length=19))
# threshold = 1000.0

# for realization in realizations
#     chosen_dict = JLD2.load("$(posteriors_directory)/R_$(realization)_Posterior_Dict.jld2", "post_data")

#     for year in years
#         # skip if this year isn't present in the dict
#         haskey(chosen_dict, year) || continue

#         A = chosen_dict[year]  # iters × params (or params × iters)
#         pnames = Symbol[:vmThresh, :fricExp, :mu_scale, :stiff_scale, :gamma0, :melt_flux]

#         if size(A,2) != length(pnames) && size(A,1) == length(pnames)
#             @warn "Transposing A; it seems to be params × iters" sizeA=size(A)
#             A = A'
#         end

#         ch = Chains(reshape(A, size(A,1), size(A,2), 1), pnames; start=1, thin=1)
#         df = DataFrame(MCMCChains.summarystats(ch; sections=:parameters))  # has :ess

#         if any(df.ess .< threshold)
#             println("Realization $(realization) — ", year, "  (at least one ESS < $(threshold))")
#             # Print ESS “as is” for this realization+year
#             for row in eachrow(df)
#                 @printf("%-14s  ESS=%g\n", String(row.parameters), row.ess)
#             end
#             println()
#         end
#     end
# end

In [115]:
# realization = 4
# years = collect(range(2030, step=15, length=19))

# # for realization in realizations
# for year in years
#     chosen_dict = JLD2.load("$(posteriors_directory)/R_$(realization)_Posterior_Dict.jld2", "post_data")

#     A = chosen_dict[year]  # iters × params
#     pnames = [:vmThresh, :fricExp, :mu_scale, :stiff_scale, :gamma0, :melt_flux]
#     if size(A,2) != length(pnames) && size(A,1) == length(pnames)
#         @warn "Transposing A; it seems to be params × iters" sizeA=size(A)
#         A = A'
#     end

#     ch  = Chains(reshape(A, size(A,1), size(A,2), 1), pnames; start=1, thin=1)

#     stats = MCMCChains.summarystats(ch; sections=:parameters)  # ChainDataFrame with mean,std,mcse,ess,rhat
#     df    = DataFrame(stats)

#     println("Realization $(realization) — ", year)
#     for row in eachrow(df)
#         @printf("%-14s  ESS=%g  mean=%g  sd=%g\n",
#                 String(row.parameters), row.ess, row.mean, row.std)
#     end
# end

In [116]:
################################################################################
################################################################################
################################################################################

#PARAMETRIC LEARNING PLOTS

################################################################################
################################################################################
################################################################################

In [117]:
# function get_up_low(post_dict, upper_quant, lower_quant)
#     p = [upper_quant,lower_quant]
#     quantiles = zeros(19, 6, 2)
#     sorted_keys = sort(collect(keys(post_dict)))
#     for i in 1:19
#         mat = post_dict[ sorted_keys[i] ]
#         quantiles[i, 1,:] = quantile( mat[:,1], p )
#         quantiles[i, 2,:] = quantile( mat[:,2], p )
#         quantiles[i, 3,:] = quantile( mat[:,3], p )
#         quantiles[i, 4,:] = quantile( mat[:,4], p )
#         quantiles[i, 5,:] = quantile( mat[:,5], p )
#         quantiles[i, 6,:] = quantile( mat[:,6], p )
#     end
#     return quantiles
# end

In [118]:
# #CHANGE THIS LINE TO WHERE THE POSTERIORS ARE LOCATED
# path_to_posteriors = "Data/Posterior_Data_New_All"

# realizations = [i for i in 1:100];

# total_widths = zeros(length(realizations),19,6)
# sorted_keys = collect(range(2030,step=15,length=19))

# #Plot each parameter's learning against last year observed, for each trajectory
# for (iter, r) in enumerate(realizations)
#     post = JLD2.load("$(path_to_posteriors)/R_$(r)_Posterior_Dict.jld2", "post_data")
#     cred_int_90_up_low = get_up_low(post, 0.95, 0.05)
#     for i in 1:2
#         for j in 1:3
#             var_idx = tuple_to_idx(i,j)
#             cred_int_90_upper = (var_idx == 5) ? log.(cred_int_90_up_low[:,var_idx,1]) : cred_int_90_up_low[:,var_idx,1]
#             cred_int_90_lower = (var_idx == 5) ? log.(cred_int_90_up_low[:,var_idx,2]) : cred_int_90_up_low[:,var_idx,2]
#             widths = cred_int_90_upper .- cred_int_90_lower
#             total_widths[iter, :, var_idx] = widths
#         end
#     end
# end

# un_cred_int_68_up_low = zeros(19,6,2)
# for i in 1:19
#     quant = [0.84, 0.16]
#     for j in 1:6
#         un_cred_int_68_up_low[i,j,:] = quantile(total_widths[:,i,j], quant)
#     end
# end

In [119]:
# output_file_name = "Plots/Parametric_Learning_Plots/parametric_learning_without_trajectories.pdf"

# title_dict = Dict([
#             1 => L"\sigma_{max}", 2 => L"q" , 3 => L"C_{\mu}" ,
#             4 => L"C_{\phi}", 5 => L"\log(\gamma_0)", 6 => L"\overline{m}" ])

# tick_dict  = Dict(1=>[22000,26000,30000,34000,38000],
#                   2=>[0.02,0.06,0.10,0.14,0.18],
#                   3=>[0.05,0.10,0.15,0.20,0.25],
#                   4=>[0.06,0.10,0.14,0.18,0.22],
#                   5=>[0.0,0.5,1.0,1.5,2.0,2.5,3.0],
#                   6=>[16,20,24,28,32])

# ylim_dict = Dict([1 => [2.1e4,3.85e4], 
#                   2 => [0.00,0.19], 
#                   3 => [0.04,0.26],
#                   4 => [0.045,0.235], 
#                   5 => [-0.1,2.8], 
#                   6 => [14.75,32.75]])

# selections = [13,26,39,44,49,70,78]

# fig, ax = PyPlot.subplots(nrows=2, ncols=3, figsize=(18, 11), dpi=300,
#                      gridspec_kw=Dict("height_ratios"=> [1, 1], "width_ratios" => [1, 1, 1],
#                                     "wspace"=> 0.175, "hspace"=> 0.25))
# FONTSIZE=20.5

# #Plot each parameter's average learning across all the trajectories
# for i in 1:2
#     for j in 1:3
#         var_idx = tuple_to_idx(i,j)
        
#         current = total_widths[:,:,var_idx]
#         mean_widths = vec(mean(current, dims=1))
#         upper = un_cred_int_68_up_low[:,var_idx,1]
#         lower = un_cred_int_68_up_low[:,var_idx,2]
#         ax[i,j].plot(sorted_keys, mean_widths, color="green",lw=2.5,label="Mean", zorder = 601)
#         ax[i,j].fill_between(sorted_keys, upper, lower, color="palegreen",alpha = 0.3, label="68% credible interval",zorder = 600)

#         #UNCOMMENT LINE BELOW TO ADD TRAJECTORIES TO PLOTS
#         for iter in selections
#             # ax[i,j].plot(sorted_keys, total_widths[iter, :, var_idx], marker = "o", markersize= 4, 
#             #             label = iter == selections[1] ? "Individual scenario" : "_nolegend_",ls="dashed", 
#             #             color = "black", markerfacecolor="green")
#         end
        
#         ax[i,j].set_title(title_dict[var_idx],fontsize=1.2*FONTSIZE, pad=10)
#         ax[i,j].ticklabel_format(style="sci", scilimits=(-2,4), useMathText=true)
#         ax[i,j].tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
#         ax[i,j].xaxis.offsetText.set_fontsize(FONTSIZE*0.8)
#         ax[i,j].yaxis.offsetText.set_fontsize(FONTSIZE*0.8)
#         ax[i,j].set_yticks(tick_dict[var_idx])
#         # ax[i,j].set_ylim(ylim_dict[var_idx])
#         ax[i,j].set_xlim([2030,2300])
#         # ax[i,j].locator_params(tight=true, nbins=6)
#         ax[i,j].grid(true, alpha=0.5, zorder=1)

#     end
# end

# handles, labels = ax[1,1].get_legend_handles_labels()
# n = length(handles)
# desired_indices = filter(i -> 1 ≤ i ≤ n, [1,2,3]) 
# sorted_handles = [handles[i] for i in desired_indices]
# sorted_labels = [labels[i] for i in desired_indices]
# fig.legend(sorted_handles, sorted_labels, loc="lower center", ncol=3, fontsize=0.8*FONTSIZE,
#            bbox_to_anchor=(0.512, 0.92), bbox_transform=PyPlot.gcf().transFigure,
#            frameon="True", framealpha=1)

# fig.supxlabel("Analysis year", fontsize=FONTSIZE*1.2, y=0.036)
# fig.supylabel("90% credible interval width", fontsize=FONTSIZE*1.2, x=0.066)

# plt.savefig(output_file_name)

In [120]:
################################################################################
################################################################################
################################################################################

# SLR PCA PROJECTION

################################################################################
################################################################################
################################################################################

In [121]:
# gps, PC_min, PC_max, p, num_pcs = JLD2.load("VAF_PCA_GP_Emulators.jld2", 
#     "gps", "PC_min", "PC_max", "p", "num_pcs");

In [122]:
# project(pca, Y, J=size(Y,1)) = (projection(pca)[:,1:J])' * centralize(Y, pca.mean)
# reconstruct(pca, W, J=size(W,1)) = decentralize(projection(pca)[:,1:J] * W, mean(pca))

In [145]:
y = CSV.read("../Data/Training_Data/Amery_VAF_time_series_2300_ssp_5_8.5.csv", DataFrame)
y = select(y, Not(1))
y = Matrix(y)
y = collect(y');

In [146]:
x = CSV.read("../Data/Training_Data/Amery_GP_Emulator_RELX_Ensemble_Filtered.csv", DataFrame);

In [147]:
vmThresh_min, vmThresh_max = minimum(x.vmThresh), maximum(x.vmThresh)
scaler_vmThresh = fit(UnitRangeTransform, x.vmThresh)
d_01 = StatsBase.transform(scaler_vmThresh, x.vmThresh)

fricExp_min, fricExp_max = minimum(x.fricExp), maximum(x.fricExp)
scaler_fricExp = fit(UnitRangeTransform, x.fricExp)
d_02 = StatsBase.transform(scaler_fricExp, x.fricExp)

mu_scale_min, mu_scale_max = minimum(x.mu_scale), maximum(x.mu_scale)
scaler_mu_scale = fit(UnitRangeTransform, x.mu_scale)
d_03 = StatsBase.transform(scaler_mu_scale, x.mu_scale)

stiff_scale_min, stiff_scale_max = minimum(x.stiff_scale), maximum(x.stiff_scale)
scaler_stiff_scale = fit(UnitRangeTransform, x.stiff_scale)
d_04 = StatsBase.transform(scaler_stiff_scale, x.stiff_scale)

gamma0_min, gamma0_max = minimum(x.gamma0), maximum(x.gamma0)
scaler_gamma0 = fit(UnitRangeTransform, x.gamma0)
d_05 = StatsBase.transform(scaler_gamma0, x.gamma0)

melt_flux_min, melt_flux_max = minimum(x.melt_flux), maximum(x.melt_flux)
scaler_melt_flux = fit(UnitRangeTransform, x.melt_flux)
d_06 = StatsBase.transform(scaler_melt_flux, x.melt_flux)

param_mins = [vmThresh_min,fricExp_min,mu_scale_min,stiff_scale_min,gamma0_min,melt_flux_min]
param_maxs = [vmThresh_max,fricExp_max,mu_scale_max,stiff_scale_max,gamma0_max,melt_flux_max]

design_scaled = DataFrame(vmThresh_scaled=d_01, fricExp_scaled=d_02, mu_scale_scaled=d_03,
                          stiff_scale_scaled=d_04, gamma0_scaled=d_05, melt_flux_scaled=d_06)

X = Matrix(design_scaled)' # matrix 6 x N

6×119 adjoint(::Matrix{Float64}) with eltype Float64:
 0.407891   0.584329  0.879853  0.35452   …  0.89069    0.435987   0.727361
 0.0356264  0.509296  0.437061  0.789839     0.0620034  0.412618   0.840317
 0.354441   0.449709  0.200242  0.311622     0.394692   0.0504651  0.712968
 0.485306   0.770992  0.664396  0.733916     0.555406   0.342915   0.709687
 0.607262   0.45634   0.784528  0.921009     0.959423   0.688045   0.451809
 0.91086    0.239113  0.835963  0.482451  …  0.11399    0.180736   0.683243

In [148]:
present2015_post_mm_ssp5 = JLD2.load("../Data/Projection_Data/2015_SLR_Projections.jld2", "present2015_post_mm_ssp5");

In [149]:
# # Function that projects sea level contribution for the sequence of calibrated posteriors 2030-2300 for a given realization(trajectory)
# function sequential_prediction_func(
#         posterior_dict::Dict{Int64, Matrix{Float64}},
#         realization::String,
#         num_pcs,
#         min_PC,
#         max_PC,
#         X_mins,
#         X_maxs,
#         fit,
#         vfa,
#         emulators

#     )
        
#         sample_size = 5000
#         projection_dict = Dict{Int64, Matrix{Float64}}()
#         #Project forward for each posterior in dictionary
        
#         for key in keys(posterior_dict)
#             #println(key)
#             post_full = DataFrame(vmThresh_post=posterior_dict[key][:,1], fricExp_post=posterior_dict[key][:,2], 
#             mu_scale_post=posterior_dict[key][:,3], stiff_scale_post=posterior_dict[key][:,4], 
#             gamma0_post=posterior_dict[key][:,5], melt_flux_post=posterior_dict[key][:,6])

#             Random.seed!(11)
#             # Get the total number of rows in the DataFrame
#             total_rows = nrow(post_full)
#             # Generate random indices to select rows
#             random_indices = randperm(total_rows)[1:sample_size]
#             # Select the subset of rows using the random indices
#             post = post_full[random_indices, :]

#             #Initialize empty arrays for saving
#             mu_post = [Vector{Float64}() for _ in 1:num_pcs]
#             sig_post = [Vector{Float64}() for _ in 1:num_pcs]
            
#             #Predict with each PC emulator for samples of the posterior
#             for x in eachrow(post)
#                 #scale parameters
#                 p1_t = (x.vmThresh_post- X_mins[1])/(X_maxs[1]-X_mins[1])
#                 p2_t = (x.fricExp_post- X_mins[2])/(X_maxs[2]-X_mins[2])
#                 p3_t = (x.mu_scale_post- X_mins[3])/(X_maxs[3]-X_mins[3])
#                 p4_t = (x.stiff_scale_post - X_mins[4])/(X_maxs[4]-X_mins[4])
#                 p5_t = (x.gamma0_post- X_mins[5])/(X_maxs[5]-X_mins[5])
#                 p6_t = (x.melt_flux_post- X_mins[6])/(X_maxs[6]-X_mins[6])
                
#                 α = [p1_t;; p2_t;; p3_t;; p4_t;; p5_t;; p6_t]'
                
#                 #Predict on each PC
#                 for i in 1:num_pcs
#                     μ, σ = predict_gp_eml(emulators[i], α)
#                     mu =  only(μ)*(max_PC[i] - min_PC[i]) + min_PC[i]
#                     sig = only(σ)*(max_PC[i] - min_PC[i])
#                     append!(mu_post[i], mu)
#                     append!(sig_post[i], sig)
#                 end
#             end

#             #Save the Principal components predictions 
#             PC_mean_pred_post = hcat(mu_post...)
#             PC_sig_pred_post = hcat(sig_post...)

#             # mkpath("../../SLR_Projection_Data/pred_mean_PC/R_$(realization)")
#             # mkpath("../../SLR_Projection_Data/pred_std_PC/R_$(realization)")
#             # @save "../../SLR_Projection_Data/pred_mean_PC/R_$(realization)/$(realization)-year$(key)mean_PCs.jld2" PC_mean_pred_post
#             # @save "../../SLR_Projection_Data/pred_std_PC/R_$(realization)/$(realization)-year$(key)std_PCs.jld2" PC_sig_pred_post

#             sample_post = Matrix{Float64}(undef, 0, size(vfa, 1))

#             #Reconstruct the PC predictions to get VAF outputs
#             for (mu_sample, sigma_sample) in zip(eachrow(PC_mean_pred_post), eachrow(PC_sig_pred_post))
#                 # Appendix B.2 steps combined to generate multivariate normal back tranformed sample
#                 sample = reconstruct(fit, rand.(Normal.(mu_sample,sigma_sample)), num_pcs)
#                 sample_post = [sample_post; sample']
#             end
#             sample_post = sample_post'
#             sample_post_mm_ssp5 = sample_post./(-362)
#             projection_dict[key] = sample_post_mm_ssp5
#             mkpath("Data/Projection_Data/R_$(realization)")
#             @save "Data/Projection_Data/R_$(realization)/$(realization)-year$(key)pred_VAF.jld2" sample_post_mm_ssp5
         

#         end
    
#         return nothing
#     end

In [150]:
# ## Use this cell to project foward for a set of trajectories of posteriors

# #Realization numbers
# Realizations = [i for i in 1:100];
# r_string = [string(i) for i in Realizations]

# yrs_all = collect(range(2030, step=15, length=19));

# for r in r_string
#     θ_posterior_fut = JLD2.load("Data/Posterior_Data_New_All/R_$(r)_Posterior_Dict.jld2", "post_data");
#     sequential_prediction_func(
#         θ_posterior_fut,
#         r,
#         5,
#         PC_min,
#         PC_max,
#         param_mins,
#         param_maxs,
#         p,
#         y,
#         gps)
#     println("Done with realization $(r)...")
# end

In [151]:
################################################################################
################################################################################
################################################################################

# SLR LEARNING PLOTS

################################################################################
################################################################################
################################################################################

In [152]:
SIZE=21
FONTSIZE=21
DOTCOLOR="green"
EDGECOLOR="gray"
plt.style.use(["default","science","no-latex"])

In [153]:
#Returns means, upper, and lower quntailes for a single year's prediction from a single (R, yr_calibrated) pair
function calculate_quantiles_mean(sample, row)
    slice = sample[row, :]
    mean_data = mean(slice)
    quantile_5 = quantile(slice, 0.05)
    quantile_95 = quantile(slice, 0.95)
    
    return mean_data, quantile_5, quantile_95

end

calculate_quantiles_mean (generic function with 1 method)

In [154]:
all_years_no_gap = collect(range(2016, step=1, length=285))
yrs_gapped = collect(range(2030, step=15, length=19));

yrs_dict = Dict{Int64, Int64}()
for (idx,yr) in enumerate(all_years_no_gap)
        yrs_dict[yr] = idx
end

In [155]:
# function plot_trajectorys(iter, selections, axis, x_values, y_values, present_value, face_color)
#     all_y = vcat([present_value], y_values)
#     axis.plot(x_values, all_y, marker = "o", markersize= 6.6, color="black", ls="dashed",
#         markerfacecolor=face_color, label = iter == selections[1] ? "Individual scenario" : "_nolegend_")
 
# end

In [156]:
# ### FIGURE 4 Plotting Script
# #Change this path to "Data/Projection_Data" once you have run the previous PCA Projection setting on your posteriors
# path_to_projections = "Data/Projection_Data"

# #Realization numbers
# realizations = [string(i) for i in 1:100];

# color_dict = Dict(
#                 2100 => "darkblue", 2200=> "chocolate", 2300=>"purple"
# )

# fill_color_dict = Dict(
#                 2100 => "dodgerblue", 2200=> "orange", 2300=>"plum"
# )

# xticks_dict = Dict(2100 => [2015,2030,2045,2060,2075,2090], 
#                    2200 => [2015,2045,2075,2105,2135,2165,2195], 
#                    2300 => [2015,2050,2100,2150,2200,2250,2300])

# yticks_dict = Dict(2100 => [3,3.5,4,4.5,5,5.5,6],
#                    2200 => [10,20,30,40,50,60,70],
#                    2300 => [10,20,30,40,50,60,70,80])

# ylim_dict = Dict(2100 => [2.84,6.32],
#                  2200 => [6,76],
#                  2300 => [6,88])

# selections = [13,27,40,45,58,70,78]



# # fig, ax = subplots(nrows=1, ncols=1, figsize=(10.5, 8))
# fig, axes = subplots(nrows=1, ncols=3, figsize=(32, 6))
# fig.subplots_adjust(wspace=0.14, hspace=0.0) 

# FONTSIZE=26
# # output_file_name = "Plots/SLR_Learning_Plots/With_Trajectories/yr_$(chosen_year)_slr_learning.pdf"
# # output_file_name = "Plots/SLR_Learning_Plots/slr_learning_without_trajectories.pdf"

# letters = ["a", "b", "c"]
# for (a,chosen_year) in enumerate([2100, 2200, 2300])
#     ax = axes[a]

#     #Corresponding row in the matrix of projections
#     row_idx = yrs_dict[chosen_year]
#     n_cal_years = fld(chosen_year-2015,15) + 1
#     full_yr_range = collect(range(2015, step=15, length= n_cal_years ))
    
#     #Get 2015 projections upper and lower quantile bounds
#     present2015_post_mm_ssp5 = JLD2.load("2015_SLR_Projections.jld2", "present2015_post_mm_ssp5")
#     present_mu, present_lower, present_upper, = calculate_quantiles_mean(present2015_post_mm_ssp5, row_idx)
#     present_width = present_upper - present_lower
    
#     #Vector to hold the widths of each trajectory, for averaging purposes later
#     total_widths = zeros(100,length(full_yr_range))
#     for (iter,realization) in enumerate(realizations)
#         one_trajectorys_widths = Vector{Float64}()
#         #loop through the years at which calibrations occur
#         for yr in full_yr_range[2:end]
#             #load projections matrix 
#             vaf_sample = JLD2.load("$(path_to_projections)/R_$(realization)/$(realization)-year$(yr)pred_VAF.jld2",
#                 "sample_post_mm_ssp5")
#             mu, lower, upper, = calculate_quantiles_mean(vaf_sample, row_idx)
#             push!(one_trajectorys_widths, upper - lower)
#         end
#         #UNCOMMENT LINE BELOW TO ADD TRJAECTORIES TO PLOT
#         if iter in selections
#             # plot_trajectorys(iter, selections, ax, full_yr_range, one_trajectorys_widths, present_width, color_dict[chosen_year])
#         end
#         total_widths[iter,:] = vcat(present_width, one_trajectorys_widths)
#     end

#     #Calculate averages and standard deviations across trajectories
#     mean_widths = vec( mean(total_widths, dims=1) )
#     std_widths = vec( std(total_widths, dims=1) )
#     upper = mean_widths .+ std_widths
#     lower = mean_widths .- std_widths
#     ax.plot(full_yr_range, mean_widths, color=color_dict[chosen_year],lw=2.5,label="Mean", zorder = 500)
#     ax.fill_between(full_yr_range, upper,lower, color=fill_color_dict[chosen_year],alpha = 0.3, label="68% credible interval", zorder = 499)
    
#     ax.set_xlabel("Analysis year", fontsize= FONTSIZE*0.8, labelpad=10)
#     if a == 1
#         ax.set_ylabel("90% credible interval width (mm)", fontsize= FONTSIZE*0.8, labelpad=12)
#     end
#     ax.tick_params(axis="both", which="major", labelsize=FONTSIZE*0.65)
#     ax.set_xlim(2015,full_yr_range[n_cal_years])
#     ax.set_ylim(ylim_dict[chosen_year])
#     ax.set_xticks(xticks_dict[chosen_year])
#     ax.set_yticks(yticks_dict[chosen_year])
#     ax.grid(true, alpha=0.5, zorder=1)
    
#     handles, labels = ax.get_legend_handles_labels()
#     n = length(handles)
#     desired_indices = filter(i -> 1 ≤ i ≤ n, [1,2,3]) 
#     sorted_handles = [handles[i] for i in desired_indices]
#     sorted_labels = [labels[i] for i in desired_indices]
#     if a == 1
#         ax.text(-0.13, 1.08, "(" * letters[a] * ")", ha="left", va="top", transform=ax.transAxes, fontsize=FONTSIZE)
#     else
#         ax.text(-0.1, 1.08, "(" * letters[a] * ")", ha="left", va="top", transform=ax.transAxes, fontsize=FONTSIZE)
#     end
#     ax.legend(sorted_handles, sorted_labels, loc="lower center", ncol=1, fontsize=FONTSIZE*0.65,
#         # bbox_to_anchor=(0.772, 0.712), frameon="True", framealpha=1)
#        bbox_to_anchor=(0.768, 0.76), frameon="True", framealpha=1)
# end

# mkpath(dirname(output_file_name))
# fig.savefig(output_file_name; bbox_inches="tight", dpi=300) 

In [157]:
################################################################################
################################################################################
################################################################################

# SLR SENSITIVITY PLOTS

################################################################################
################################################################################
################################################################################

In [158]:
# --- inputs you already have ---
path_to_posteriors = "../Data/Posterior_Data"
future_obs_directory = "../Data/Future_Observation_Data/Generative_Parameters/Official_Constraining_Observations-metadata"
output_file_name   = "../Plots/Sensitivity_Plots/SLR_sensitivity_wrt_parameters.pdf"
path_to_projections = "../Data/Projection_Data"

chosen_year =  2300
row_idx = yrs_dict[chosen_year]

# Realization numbers
realizations = [string(i) for i in 1:100];

# --- collect widths for all realizations ---
widths = Array{Float64}(undef, length(realizations))
θ_mat  = Array{Float64}(undef, length(realizations), 6)

for (i, r) in enumerate(realizations)
    vaf_sample = JLD2.load(joinpath(path_to_projections, "R_$(r)", "$(r)-year$(chosen_year)pred_VAF.jld2"),
                           "sample_post_mm_ssp5")
    # no trailing comma
    μ, lower, upper = calculate_quantiles_mean(vaf_sample, row_idx)

    widths_all_years = upper .- lower          # elementwise
    widths[i] = widths_all_years[end]          # last year’s width (or use row_idx if that’s per-year already)

    θ = JLD2.load(joinpath(future_obs_directory, "$(r)_emulator_data.jld2"), "θ")
    θ_mat[i, :] = unscale_params(θ)            # ensure this returns a 6-vector
end

In [159]:
# # --- plotting: 2×3 grid, one subplot per parameter ---
# fig, ax = PyPlot.subplots(nrows=2, ncols=3, figsize=(18, 11), dpi=300,
#     gridspec_kw=Dict("height_ratios"=>[1,1], "width_ratios"=>[1,1,1], "wspace"=>0.175, "hspace"=>0.25))

# FONTSIZE=20.5
# # fig.suptitle("Sea-level Contributions 90% Credible Interval Widths at $(chosen_year)"; fontsize=30, fontweight="bold", y=0.98)

# title_dict = Dict(1=>L"\sigma_{max}", 2=>L"q", 3=>L"C_{\mu}", 4=>L"C_{\phi}", 5=>L"\log(\gamma_0)", 6=>L"\overline{m}")
# tick_dict  = Dict(1=>[1.1e5,1.2e5,1.3e5,1.4e5],
#                   2=>[0.15,0.2,0.25,0.3],
#                   3=>[0.8,0.9,1,1.1],
#                   4=>[0.82,0.88,0.94,1,1.06],
#                   5=>[9.17,10,11,12],
#                   6=>[12,20,30,40,50,58])

# xlim_dict = Dict([1 => [1.02e5,1.485e5], 
#                   2 => [0.108,0.333], 
#                   3 => [0.78,1.148],
#                   4 => [0.81,1.095], 
#                   5 => [9.05,12.6], 
#                   6 => [10,60]])

# for j in 1:6
#     rI = (j-1) ÷ 3 + 1
#     cI = (j-1) % 3 + 1
#     axi = ax[rI, cI]

#     if j == 5
#         var = np.log(θ_mat[:, j])
#     else
#         var = θ_mat[:, j]
#     end

#     seaborn.regplot(x=var, y=widths, lowess=true,
#               scatter_kws=Dict("alpha"=>0.85, "s"=> SIZE*2, "color"=>DOTCOLOR, 
#                                 "edgecolor"=>EDGECOLOR, "linewidths"=>0.5, 
#                                 "plotnonfinite"=>false, "zorder"=>2), 
#               line_kws=Dict("color" => "red", "lw" => 1, "alpha" => 0.8), ax=axi)

    
#     axi.set_title(string(title_dict[j]), fontsize=1.2*FONTSIZE, pad=10)
#     axi.ticklabel_format(style="sci", scilimits=(-2,2), useMathText=true)
#     axi.tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
#     axi.xaxis.offsetText.set_fontsize(FONTSIZE*0.8)
#     axi.yaxis.offsetText.set_fontsize(FONTSIZE*0.8)
#     axi.set_xticks(tick_dict[j])
#     axi.set_xlim(xlim_dict[j])
#     axi.set_ylim([8.51,22])
#     axi.set_yticks([10,15,20])
#     axi.locator_params(tight=true, nbins=6)
#     axi.grid(true, alpha=0.5, zorder=1)
# end

# mkpath(dirname(output_file_name))
# fig.savefig(output_file_name; bbox_inches="tight", dpi=300)   # should now complete quickly
# PyPlot.close(fig)

In [160]:
# # Assumptions you already have:
# # - θ_mat :: Matrix{Float64}  (n_realizations × 6)   # parameters per realization
# # - widths :: Vector{Float64}  (n_realizations)      # 90% CI width at chosen_year
# # - FONTSIZE, SIZE, EDGECOLOR defined
# # - title_dict, tick_dict like in your snippet (LaTeXStrings are fine)
# cmap = mpl.cm.get_cmap("viridis")

# output_file_name   = "Plots/Sensitivity_Plots/SLR_sensitivity_wrt_pairwise_parameters.pdf"

# # Choose a sequential cmap and robust color limits for widths (since widths ≥ 0)

# vmin, vmax = quantile(widths, [0.05, 0.95])   # robust limits; change to [minimum(widths), maximum(widths)] if you prefer
# norm_ = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

# # Build all 15 unique pairs (1..6 choose 2), row-major ordering
# pairs = [(a,b) for a in 1:6 for b in a+1:6]  # length = 15

# # Figure: 5 rows × 3 cols
# fig, ax = PyPlot.subplots(nrows=5, ncols=3, figsize=(22, 29), width_ratios=[1,1,1], height_ratios=[1,1,1,1,1])
# # fig.suptitle("Sea-level Contributions 90% Credible Interval Widths at $(chosen_year)"; fontsize=30, fontweight="bold", y=1.008)

# # If ax is a Matrix{PyObject}, indexing is ax[row, col]
# for (idx, (xk, yk)) in enumerate(pairs)
#     rI = 1 + (idx-1) ÷ 3
#     cI = 1 + (idx-1) % 3
#     axi = ax[rI, cI]

#     # Scatter: x = param xk, y = param yk, color = widths
#     x_var = (xk == 5) ? log.(θ_mat[:, xk]) : θ_mat[:, xk]
#     y_var = (yk == 5) ? log.(θ_mat[:, yk]) : θ_mat[:, yk]

#     axi.set_rasterization_zorder(0)
#     sc = axi.scatter(x_var, y_var;
#                      s=SIZE*5, c=widths, cmap=cmap, 
#                      edgecolor=EDGECOLOR, linewidth=0.5,
#                      rasterized=true)

#     # Labels/ticks/limits
#     axi.set_xlabel(string(title_dict[xk]), fontsize=FONTSIZE, labelpad=10)
#     axi.set_ylabel(string(title_dict[yk]), fontsize=FONTSIZE, labelpad=10)
#     axi.ticklabel_format(style="sci", scilimits=(-3,3), useMathText=true)
#     axi.tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
#     axi.xaxis.offsetText.set_fontsize(FONTSIZE*0.8)
#     axi.yaxis.offsetText.set_fontsize(FONTSIZE*0.8)
#     axi.locator_params(tight=true, nbins=6)
#     axi.grid(true, alpha=0.5, zorder=1)
#     axi.set_xticks(tick_dict[xk])
#     axi.set_yticks(tick_dict[yk])
#     axi.set_xlim(xlim_dict[xk])
#     axi.set_ylim(xlim_dict[yk])
# end

# # Single shared colorbar on the right
# plt.tight_layout(rect=[0.0, 0.0, 0.95, 1.0])  # leave room for colorbar
# cax = fig.add_axes([0.98, 0.25, 0.02, 0.5])  # [left, bottom, width, height]
# sm = mpl.cm.ScalarMappable(norm=norm_, cmap=cmap); sm.set_array([])
# cbar = fig.colorbar(sm, cax=cax)
# cbar.ax.set_title("90% \ncredible \ninterval \nwidth", fontsize=FONTSIZE, pad=30)
# cbar.ax.tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
# cbar.mappable.set_clim(8.5, 22)

# mkpath(dirname(output_file_name))
# fig.savefig(output_file_name; bbox_inches="tight", dpi=300)

In [161]:
################################################################################
################################################################################
################################################################################

# NATHAN PAPER PLOTS

################################################################################
################################################################################
################################################################################

In [162]:
present2015_post_mm_ssp5 = JLD2.load("../Data/Projection_Data/2015_SLR_Projections.jld2", "present2015_post_mm_ssp5");

In [163]:
function plot_param_means_of_trajectory(real, posterior_dict, axis, title_dict, tick_dict, ylim_dict, color, fill_color, fontsize_param)
    FONTSIZE = fontsize_param
    # Years at which recalibration occurred
    cal_years = collect(range(2015, step=15, length=19))
    means = Dict(j => Vector{Float64}() for j in 1:6)
    uppers = Dict(j => Vector{Float64}() for j in 1:6)
    lowers = Dict(j => Vector{Float64}() for j in 1:6)

    for j in 1:6
        current_post = θ_posterior_2015
        var = (j == 5) ? log.(current_post[:, j]) : current_post[:, j]
        push!(means[j], mean(var))
        push!(uppers[j], quantile(var, 0.95))
        push!(lowers[j], quantile(var, 0.05))
    end

    for yr in cal_years[2:end]
        current_post = posterior_dict[yr]
        for j in 1:6
            var = (j == 5) ? log.(current_post[:, j]) : current_post[:, j]
            push!(means[j], mean(var))
            push!(uppers[j], quantile(var, 0.95))
            push!(lowers[j], quantile(var, 0.05))
        end
    end

    for j in 1:6
        rI = (j-1) ÷ 2 + 1
        cI = (j-1) % 2 + 1
        ax = axis[rI, cI]
        # ax = axis[j]
        ax.spines["top"].set_visible(false)
        ax.spines["right"].set_visible(false)
        ax.tick_params(which="both", top=false, right=false, labeltop=false, labelright=false)
        ax.plot(cal_years,means[j],label="Parameter posterior distribution mean",zorder=20, color=color, lw=3)
        ax.fill_between(cal_years, uppers[j],lowers[j],alpha = 0.5,label="90% credible interval", color=fill_color)
        ax.ticklabel_format(style="sci", scilimits=(-2,4), useMathText=true)
        ax.tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
        ax.xaxis.offsetText.set_fontsize(FONTSIZE*0.8)
        ax.yaxis.offsetText.set_fontsize(FONTSIZE*0.8)
        if rI == 3
            ax.set_xlabel("Analysis year", fontsize=FONTSIZE, labelpad=10)
        end
        ax.set_ylabel(title_dict[j], fontsize=FONTSIZE, labelpad=10)
        ax.set_yticks(tick_dict[j])
        ax.set_ylim(ylim_dict[j])
        ax.set_xlim(2015,2305)
        xticks = rI == 3 ? [2015,2050,2100,2150,2200,2250,2300] : []
        ax.set_xticks(xticks)
        # ax.locator_params(tight=true, nbins=6)
        ax.grid(true, alpha=0.25, zorder=1)

        y_true = (j == 5) ? log.(θ_mat[real, j]) : θ_mat[real, j]
        dx = 5        # small nudge to the right
        ax.scatter([cal_years[end]+dx], [y_true];
           marker="D", s=64, c=color, edgecolors="k", linewidths=0.8, zorder=21, clip_on=false)
    end  
end

plot_param_means_of_trajectory (generic function with 1 method)

In [164]:
all_years_no_gap = collect(range(2016, step=1, length=285))
yrs_gapped = collect(range(2030, step=15, length=19));

yrs_dict = Dict{Int64, Int64}()
for (idx,yr) in enumerate(all_years_no_gap)
        yrs_dict[yr] = idx
end

In [167]:
function plot_slr_means_of_trajectory(path_to_proj, axis, r, pred_year, color, fill_color, fontsize_slr)
    FONTSIZE = fontsize_slr
    # Years at which recalibration occurred
    cal_years = collect(range(2015, step=15, length=20))
    # @show cal_years
    # Maximum number of calibration years that occurred before year projected
    n_cal_years = fld(pred_year-2015,15) + 1
    
    means = Vector{Float64}()
    uppers = Vector{Float64}()
    lowers = Vector{Float64}()
    
    # Index of the matrix containing the specified years projections
    pred_idx = yrs_dict[pred_year]

    push!(means, mean(present2015_post_mm_ssp5[pred_idx,:] ))
    push!(uppers, quantile(present2015_post_mm_ssp5[pred_idx,:], 0.95))
    push!(lowers, quantile(present2015_post_mm_ssp5[pred_idx,:], 0.05))
    
    for yr in cal_years[2:n_cal_years]
        current_projections = JLD2.load("$(path_to_proj)/$(r)-year$(yr)pred_VAF.jld2","sample_post_mm_ssp5" )
        # Calculate the means and quantiles of that row 
        push!(means, mean( current_projections[pred_idx,:] ))
        push!(uppers, quantile(current_projections[pred_idx,:], 0.95))
        push!(lowers, quantile(current_projections[pred_idx,:], 0.05))

    end
    # Plot against year calibrated
    axis.plot(cal_years[1:n_cal_years], means, label="Projection distribution mean", zorder=20, color=color, lw=3)
    axis.fill_between(cal_years[1:n_cal_years], uppers, lowers,alpha=0.5, label="90% credible interval", color=fill_color)
    axis.set_xlim(2015,cal_years[n_cal_years])
    xticks = (pred_year == 2300) ? [2015,2050,2100,2150,2200,2250,2300] : cal_years[1:1+floor(Int,pred_idx/100):n_cal_years]
    axis.set_xticks(xticks)
    axis.tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
    axis.grid(true, alpha=0.25, zorder=1)
    axis.set_xlabel("Analysis year", fontsize=FONTSIZE, labelpad=10)
    axis.set_ylabel("Sea-level contribution at $(pred_year) (mm)", fontsize=FONTSIZE, labelpad=10)
end

plot_slr_means_of_trajectory (generic function with 1 method)

In [1]:
#Realization numbers
Realizations = [string(i) for i in 1:100];

Random.seed!(7);
# Only selecting 3 trajectories because the graph gets to messy with more.
selections = [14,24,49,77]
println(selections)

path_to_posteriors = "../Data/Posterior_Data"
path_to_slr_projections = "../Data/Projection_Data"


title_dict = Dict([
            1 => L"\sigma_{max}", 2 => L"q" , 3 => L"C_{\mu}" ,
            4 => L"C_{\phi}", 5 => L"\log(\gamma_0)", 6 => L"\overline{m}" ])

tick_dict  = Dict(1=>[100000,120000,140000],
                  2=>[0.15,0.20,0.25,0.30],
                  3=>[0.8,0.9,1,1.1],
                  4=>[0.8,0.9,1],
                  5=>[9.17,10,11,12],
                  6=>[20,30,40,50])

ylim_dict = Dict([1 => [0.98e5,1.5e5], 
                  2 => [0.093,0.34], 
                  3 => [0.79,1.14],
                  4 => [0.79,1.08], 
                  5 => [9.05,12.2], 
                  6 => [11.5,58.5]])

color_dict = Dict(
                1 => "red", 2=> "green", 3=>"purple", 4=>"blue"
)

fill_color_dict = Dict(
                1 => "lightcoral", 2=> "palegreen", 3=>"plum", 4=>"lightblue"
)

# Generic handles and labels for the figures
gray_patch = matplotlib.patches.Patch(color="gray", label="90% credible interval",alpha=0.5)
black_line_param = matplotlib.lines.Line2D([], [], lw=3, color="black", label="Parameter posterior distribution mean")
black_line_slr = matplotlib.lines.Line2D([], [], lw=3, color="black", label="Projection distribution mean")

# Define the figure subplots for parametric analysis
fig_params, ax_params = PyPlot.subplots(nrows=3, ncols=2, figsize=(28, 15), dpi=300,
                     gridspec_kw=Dict("height_ratios"=> [1, 1, 1], "width_ratios"=> [1, 1], "hspace"=> 0.15, "wspace"=> 0.1))
FONTSIZE_PARAM = 23
# Loop to call the parametric plotting function 
for (i,r) in enumerate(selections)
    posterior_dict = JLD2.load("$(path_to_posteriors)/R_$(r)_Posterior_Dict.jld2", "post_data")
    plot_param_means_of_trajectory(r, posterior_dict, ax_params, title_dict, tick_dict, ylim_dict, 
                                    color_dict[i], fill_color_dict[i], FONTSIZE_PARAM)
end

#Figure labels and legends
ax_params[end].set_xlabel("Analysis year", fontsize=FONTSIZE_PARAM, labelpad=10)
# fig_params.supylabel("Parameter value", fontsize=FONTSIZE)
fig_params.legend(handles=[black_line_param, gray_patch],loc="upper center", ncol=6, fontsize=0.75*FONTSIZE,
           bbox_to_anchor=(0.512, 0.92), bbox_transform=PyPlot.gcf().transFigure,
           frameon="True", framealpha=1)
param_plot_saveout_path = "../Plots/Individual_Trajectory_Learning_Plots/param_posterior_against_analysis_year.pdf"
mkpath(dirname(param_plot_saveout_path))
#fig_params.savefig(param_plot_saveout_path; dpi=300)
plt.close()

fig_slr, axes = subplots(nrows=1, ncols=3, figsize=(33, 6))
fig_slr.subplots_adjust(wspace=0.2, hspace=0.0)
slr_plot_saveout_path = "../Plots/Individual_Trajectory_Learning_Plots/projected_slr_against_analysis_year.pdf"
FONTSIZE_SLR = 23
# Loop to call the SLR plotting function for projections for three future years
letters = ["a", "b", "c"]
for (a, chosen_year) in enumerate([2100, 2200, 2300])
    ax_slr = axes[a]
    for (i,r) in enumerate(selections)
        plot_slr_means_of_trajectory("$(path_to_slr_projections)/R_$(r)", ax_slr, r, chosen_year, color_dict[i], 
                                                                            fill_color_dict[i], FONTSIZE_SLR)
    end
    if a == 1
        ax_slr.set_ylim([-3.97,4.75])
    end
    if a == 3
        ax_slr.text(-0.157, 1.11, "(" * letters[a] * ")", ha="left", va="top", transform=ax_slr.transAxes, fontsize=FONTSIZE_SLR*0.9)
    elseif a == 2
        ax_slr.text(-0.145, 1.11, "(" * letters[a] * ")", ha="left", va="top", transform=ax_slr.transAxes, fontsize=FONTSIZE_SLR*0.9)
    else
        ax_slr.text(-0.149, 1.11, "(" * letters[a] * ")", ha="left", va="top", transform=ax_slr.transAxes, fontsize=FONTSIZE_SLR*0.9)
    end
    ax_slr.legend(handles=[black_line_slr, gray_patch],loc="upper center", ncol=1, fontsize=FONTSIZE_SLR*0.66,
           bbox_to_anchor=(0.762, 1.01),frameon="True", framealpha=1)
end
mkpath(dirname(slr_plot_saveout_path))
#fig_slr.savefig(slr_plot_saveout_path; bbox_inches="tight", dpi=300)
#plt.close()

LoadError: UndefVarError: `Random` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: Random is loaded but not imported in the active module Main.